In [35]:
import quant_utils.data_moudle as dm
from quant_utils.constant import DB_CONN_JJTG_DATA
from tqdm import tqdm
import pandas as pd
from quant_utils.performance import Performance

In [59]:
def get_return_except(date):
    query_sql = f"""
    SELECT
        a.TRADE_DT,
        a.TICKER_SYMBOL,
        F_AVGRETURN_THISMONTH,
        F_AVGRETURN_THISQUARTER,
        F_AVGRETURN_QUARTER as RETURN_3M,
        F_AVGRETURN_HALFYEAR as RETURN_6M,
        F_AVGRETURN_YEAR as RETURN_1Y,
        F_AVGRETURN_TWOYEAR as RETURN_2Y,
        F_AVGRETURN_THREEYEAR as RETURN_3Y,
        F_AVGRETURN_FIVEYEAR as RETURN_5Y,
        F_AVGRETURN_YEAR - F_AVGRETURN_QUARTER AS RETURN_1Y_3M,
        F_AVGRETURN_TWOYEAR - F_AVGRETURN_QUARTER AS RETURN_2Y_3M,
        F_AVGRETURN_TWOYEAR - F_AVGRETURN_HALFYEAR AS RETURN_2Y_6M,
        F_AVGRETURN_THREEYEAR - F_AVGRETURN_QUARTER AS RETURN_3Y_3M,
        F_AVGRETURN_THREEYEAR - F_AVGRETURN_HALFYEAR AS RETURN_3Y_6M,
        F_AVGRETURN_FIVEYEAR - F_AVGRETURN_QUARTER AS RETURN_5Y_3M,
        F_AVGRETURN_FIVEYEAR - F_AVGRETURN_HALFYEAR AS RETURN_5Y_6M,
        F_AVGRETURN_FIVEYEAR - F_AVGRETURN_YEAR AS RETURN_5Y_1Y,
        F_AVGRETURN_FIVEYEAR - F_AVGRETURN_TWOYEAR AS RETURN_5Y_2Y
    FROM
        fund_performance a

    WHERE
        1 = 1 
        and a.TRADE_DT = "{date}"
    """

    ret_df = DB_CONN_JJTG_DATA.exec_query(query_sql)

    query_sql = f"""
	SELECT
        TICKER_SYMBOL,
        LEVEL_1,
        LEVEL_2,
        LEVEL_3
	FROM
	    fund_type_own 
	WHERE
        1=1
        and REPORT_DATE = (
            SELECT max(REPORT_DATE) 
                FROM fund_type_own 
            WHERE 
                1=1 
                and PUBLISH_DATE <= "{date}"
        )
    """
    fund_type = DB_CONN_JJTG_DATA.exec_query(query_sql)

    return ret_df.merge(fund_type, on="TICKER_SYMBOL", how="inner")

def shift_return(df, return_period):
    df = df.copy().sort_values(by="TRADE_DT")
    df["return_future"] = df[return_period].shift(-1)
    return df

def get_return_rank_by_type(df, return_name):
    df = df.copy()
    df = df[["TICKER_SYMBOL", return_name, "return_future"]].dropna()
    if not df.empty:
        df["history_return_group"] = pd.qcut(df[return_name], 5, labels=range(1, 6))
        df["futur_return_group"] = pd.qcut(df["return_future"], 5, labels=range(1, 6))
        return df

In [60]:
period = 'q'
dates_list = dm.get_period_end_date(start_date="20170101", end_date="20240315", period=period)
data_set = []
for date in tqdm(dates_list):
    df = get_return_except(date)
    data_set.append(df)
data_set = pd.concat(data_set)

100%|██████████| 28/28 [00:15<00:00,  1.77it/s]


In [81]:
return_period = {'q': 'F_AVGRETURN_THISQUARTER', 'm': 'F_AVGRETURN_THISMONTH'}
perf_result = []
alpha_result = []
result_decribe = []
return_name_list = [
    "RETURN_3M", "RETURN_6M", "RETURN_1Y", "RETURN_2Y", 
    "RETURN_3Y",  "RETURN_1Y_3M", "RETURN_2Y_3M", 
    "RETURN_2Y_6M", "RETURN_3Y_3M", "RETURN_3Y_6M", 
    # "RETURN_5Y",
    # "RETURN_5Y_3M", "RETURN_5Y_6M", "RETURN_5Y_1Y", "RETURN_5Y_2Y"
]
for return_name in return_name_list:
    print(return_name)
    data_set = (
        data_set.groupby('TICKER_SYMBOL')
        .apply(shift_return, return_period=return_period[period])
        .reset_index(drop=True)
    )

    target_dataset = (
        data_set.query("LEVEL_1 == '固收' and LEVEL_2=='短债' and LEVEL_3 =='短债'")
    )

    result = (
        target_dataset
        .groupby(by=["TRADE_DT"])
        .apply(get_return_rank_by_type, return_name=return_name)
        .reset_index()
    )

    result_decribe_temp = result.groupby(by=["history_return_group"])["return_future"].mean().reset_index()
    result_decribe_temp["return_name"] = return_name
    result_decribe_temp["return_period"] = period
    result_decribe.append(result_decribe_temp)

    alpha_df = result.groupby(by=["TRADE_DT", "history_return_group"])["return_future"].mean().reset_index()
    alpha_df = alpha_df.pivot_table(index="TRADE_DT", columns="history_return_group", values="return_future")
    alpha_df["alpha"] = ((alpha_df[5] - alpha_df[1])/100 + 1).cumprod() - 1
    alpha_df["alpha"] = alpha_df["alpha"]*100
    alpha_df["return_name"] = return_name
    # alpha_df["alpha"].plot(
    #     title=f"{return_period[period].split('_')[-1]}-{return_name.split('_')[1:]}/alpha"
    # )
    alpha_result.append(alpha_df)
    test_perf = Performance((alpha_df["alpha"]/100 + 1)).stats().T
    test_perf["return_name"] = return_name
    test_perf["return_period"] = period
    perf_result.append(test_perf)

RETURN_3M


C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result_decribe_temp = result.groupby(by=["history_return_group"])["return_future"].mean().reset_index()
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  alpha_df = result.groupby(by=["TRADE_DT", "history_return_group"])["return_future"].mean().reset_index()


RETURN_6M


C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result_decribe_temp = result.groupby(by=["history_return_group"])["return_future"].mean().reset_index()
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  alpha_df = result.groupby(by=["TRADE_DT", "history_return_group"])["return_future"].mean().reset_index()


RETURN_1Y


...

C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result_decribe_temp = result.groupby(by=["history_return_group"])["return_future"].mean().reset_index()
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  alpha_df = result.groupby(by=["TRADE_DT", "history_return_group"])["return_future"].mean().reset_index()


RETURN_3Y_3M


C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result_decribe_temp = result.groupby(by=["history_return_group"])["return_future"].mean().reset_index()
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  alpha_df = result.groupby(by=["TRADE_DT", "history_return_group"])["return_future"].mean().reset_index()


RETURN_3Y_6M


C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:31: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result_decribe_temp = result.groupby(by=["history_return_group"])["return_future"].mean().reset_index()
C:\Users\Wilcoxon\AppData\Local\Temp\ipykernel_20424\2821703995.py:36: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  alpha_df = result.groupby(by=["TRADE_DT", "history_return_group"])["return_future"].mean().reset_index()


In [82]:
perf_result = pd.concat(perf_result)
alpha_result = pd.concat(alpha_result)
result_decribe = pd.concat(result_decribe)

In [83]:
perf_result.sort_values(by='收益波动比', ascending=False)

,起始日期,结束日期,累计收益率,年化收益率,年化波动率,收益波动比,最大回撤,年化收益回撤比,最大回撤日,最大回撤修复,return_name,return_period
0,2017-03-31,2023-09-28,0.086831,0.013407,0.051522,0.260228,0.003866,3.467768,20221230,182,RETURN_6M,q
0,2017-03-31,2023-09-28,0.077589,0.012024,0.049451,0.243147,0.005514,2.180668,20180629,182,RETURN_1Y,q
0,2017-03-31,2023-09-28,0.086263,0.013323,0.056276,0.236738,0.004328,3.077997,20180330,91,RETURN_3M,q
0,2017-03-31,2023-09-28,0.063271,0.009861,0.055969,0.176189,0.008862,1.11272,20180928,459,RETURN_1Y_3M,q
0,2017-03-31,2023-09-28,0.043486,0.006832,0.056324,0.121294,0.011923,0.572963,20190329,824,RETURN_2Y,q
0,2017-03-31,2023-09-28,0.02387,0.00378,0.040525,0.093281,0.01159,0.326155,20190329,733,RETURN_2Y_3M,q
0,2017-03-31,2023-09-28,0.021859,0.003465,0.042477,0.081564,0.014156,0.244751,20190329,733,RETURN_2Y_6M,q
0,2017-03-31,2023-09-28,0.02152,0.003411,0.065025,0.052461,0.009725,0.350793,20190329,1189,RETURN_3Y_3M,q
0,2017-03-31,2023-09-28,0.017803,0.002826,0.065714,0.043011,0.010762,0.262635,20200630,1095,RETURN_3Y_6M,q
0,2017-03-31,2023-09-28,0.011026,0.001755,0.057295,0.030638,0.015614,0.112427,20210331,99999,RETURN_3Y,q


In [65]:
alpha_result.to_excel("d:/data.xlsx")